#### Cálculo de la Cross Correlation Bright/Faint-UHECRs

In [2]:
def generate_RandomCatalogue(ra,dec,nmult,seed):
    
    np.random.seed(seed)

    ra_min = np.min(ra)
    ra_max = np.max(ra)
    dec_min = np.min(dec)
    dec_max = np.max(dec)

    rand_ra = np.random.uniform(ra_min, ra_max, len(ra)*nmult)
    rand_sindec = np.random.uniform(np.sin(dec_min*np.pi/180.), np.sin(dec_max*np.pi/180.), \
                                    len(ra)*nmult)
    rand_dec = np.arcsin(rand_sindec)*180./np.pi

    return rand_ra, rand_dec

##########################################################

nmult = 50

minsep = 10.
maxsep = 90.
nbins = 15

##########################################################

import numpy as np
from astropy.io import ascii
from nbodykit.lab import *

# Read UHECR
filename_e = '../data/Auger/events_a8_lb.dat'
events_a8 = ascii.read(filename_e)

# Read Gxs
filename_g = '../data/VLS/2MRSxWISE_VLS.txt'
data = ascii.read(filename_g)

data_decmin, data_decmax = data['_DEJ2000'].min(),data['_DEJ2000'].max()
data_ramin, data_ramax = data['_RAJ2000'].min(),data['_RAJ2000'].max()

##################################################################################################

data_bright = data[(data['K_abs']<-23.2)&(data['K_abs']>-26.)]
data_faint = data[(data['K_abs']>-23.2)&(data['K_abs']<-22.)]

#Get min and max for RA and DEC in both samples
data_bright_decmin = data_bright['_DEJ2000'].min()
data_bright_decmax = data_bright['_DEJ2000'].max()
data_bright_ramin = data_bright['_RAJ2000'].min()
data_bright_ramax = data_bright['_RAJ2000'].max()
data_faint_decmin = data_faint['_DEJ2000'].min()
data_faint_decmax = data_faint['_DEJ2000'].max()
data_faint_ramin = data_faint['_RAJ2000'].min()
data_faint_ramax = data_faint['_RAJ2000'].max()


brightcat = ArrayCatalog(data_bright)
faintcat = ArrayCatalog(data_faint)
eventscat = ArrayCatalog(events_a8)

brightcat['RA'] = brightcat['_RAJ2000']
brightcat['DEC'] = brightcat['_DEJ2000']
faintcat['RA'] = faintcat['_RAJ2000']
faintcat['DEC'] = faintcat['_DEJ2000']
eventscat['DEC'] = eventscat['dec']


"""
Generating random catalogues
"""
rancat_b = RandomCatalog(len(brightcat)*nmult, seed=42)
rancat_b['RA']  = rancat_b.rng.uniform(low=data_bright_ramin, high=data_bright_ramax)
rancat_b['sinDEC'] = rancat_b.rng.uniform(low=np.sin(data_bright_decmin*np.pi/180.), \
                                       high=np.sin(data_bright_decmax*np.pi/180.))
rancat_b['DEC'] = np.arcsin(rancat_b['sinDEC'])*180./np.pi


rancat_f = RandomCatalog(len(faintcat)*nmult, seed=43)
rancat_f['RA']  = rancat_f.rng.uniform(low=data_faint_ramin, high=data_faint_ramax)
rancat_f['sinDEC'] = rancat_f.rng.uniform(low=np.sin(data_faint_decmin*np.pi/180.), \
                                       high=np.sin(data_faint_decmax*np.pi/180.))
rancat_f['DEC'] = np.arcsin(rancat_f['sinDEC'])*180./np.pi

#################################################################################################
print('Bright DEC range:%5.2fdeg - %5.2fdeg' % (data_bright_decmin,data_bright_decmax))
print('Bright RA range:%5.2fdeg - %5.2fdeg' % (data_bright_ramin, data_bright_ramax))

print('Faint DEC range:%5.2fdeg - %5.2fdeg' % (data_faint_decmin,data_faint_decmax))
print('Faint RA range:%5.2fdeg - %5.2fdeg' % (data_faint_decmin,data_faint_decmax))

print('Ran1 DEC range:%5.2fdeg - %5.2fdeg' % (rancat_b['DEC'].min(),rancat_b['DEC'].max()))
print('Ran1 RA range:%5.2fdeg - %5.2fdeg' % (rancat_b['RA'].min(),rancat_b['RA'].max()))
#################################################################################################

bins = np.linspace(minsep, maxsep, nbins+1)

r_cross_b = SurveyData2PCF(mode='angular',\
                        data1=brightcat, randoms1=rancat_b,\
                        data2=eventscat, randoms2=rancat_b,\
                        edges=bins, ra='RA', dec='DEC')

r_cross_f = SurveyData2PCF(mode='angular',\
                        data1=faintcat, randoms1=rancat_f,\
                        data2=eventscat, randoms2=rancat_f,\
                        edges=bins, ra='RA', dec='DEC')


Bright DEC range:-85.40deg - 44.95deg
Bright RA range: 0.81deg - 359.86deg
Faint DEC range:-87.77deg - 44.91deg
Faint RA range:-87.77deg - 44.91deg
Ran1 DEC range:-85.38deg - 44.95deg
Ran1 RA range: 0.82deg - 359.85deg


RuntimeError: calling the function 'Corrfunc.mocks.DDtheta_mocks.DDtheta_mocks' failed, likely due to issues with input data/parameters. Open at issue at https://github.com/bccp/nbodykit/issues for further help.
exception: RuntimeError occurred
stdout: 
stderr: Received signal = `Interrupt' (signo = 2). Aborting 


#### Plot

In [ ]:
import matplotlib.pyplot as plt

Nd1 = len(data_bright)
Nd2 = len(events_a8)
Nr = len(rancat_b)

pnoise_b = (1+r_cross_b.corr['corr'])/np.sqrt(r_cross_b.D1D2['npairs'])
pnoise_f = (1+r_cross_f.corr['corr'])/np.sqrt(r_cross_f.D1D2['npairs'])

plt.errorbar(r_cross_f.corr['theta'],r_cross_f.corr['corr'],yerr=pnoise_f,label='faint',c='r')
plt.errorbar(r_cross_b.corr['theta'],r_cross_b.corr['corr'],yerr=pnoise_b,label='bright',c='b')

plt.legend()
plt.xlabel(r"$\theta$ [$\mathrm{degrees}$]")
plt.ylabel(r"$\omega_{LS}$")
plt.savefig(f'../data/cross_nbodykit_bf_{nmult}.png')
plt.show()

np.savez(f'../data/nbodykit_cross_bright_{nmult}.npz',\
         r_cross_b.corr['theta'],r_cross_b.corr['corr'],pnoise_b)
np.savez(f'../data/nbodykit_cross_faint_{nmult}.npz',\
         r_cross_f.corr['theta'],r_cross_f.corr['corr'],pnoise_f)